In [1]:
%reset
%load_ext autoreload
%autoreload 2
%matplotlib notebook
#%matplotlib inline  
#%matplotlib
#matplotlib qt
#from matplotlib import pyplot as plt
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
#from mesh2D import mesh2DInclusion
from mesh2D import SquareWithInclusion
n = 2
mrl = 3
coords, emat, colorvec = SquareWithInclusion(mrl,n,[])

N = n+1;
i_perim = np.concatenate(
    (np.arange(0,N-1),
    np.arange(N-1, N**2-1, N),
    np.arange(N**2-1, N**2-N, -1),
    np.arange(N**2-n-1,N-1,-N)))

from mesh2D import plotmesh

plotmesh(coords,emat[:,i_perim],colorvec)

for i in emat[200,:]:
    plt.text(coords[i,0],coords[i,1],str(i))
    
print(emat[200,:])

In [ ]:
import numpy as np
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve
#n = np.sqrt(np.shape(emat)[1]) + 1


# fix boundaries
ifix1 = abs(0.5-abs(coords[:,0]))<1e-6
ifix2 = abs(0.5-abs(coords[:,1]))<1e-6
ifix3 = abs(np.sqrt(np.sum(coords**2,axis=1))-0.25)<1e-6
ifix = np.vstack((ifix1 | ifix2 ,ifix1 | ifix2)) | np.vstack((ifix3,ifix3))
ifix = np.reshape(ifix,(np.size(ifix),1),'F')
inotfix = np.where(np.logical_not(ifix))[0]




Kmini = np.array([[1,-1],[-1,1]])
kspring = 1
L0 = 0.1


# DOF per node
dpn = 2
    
coords2 = np.copy(coords)
n_steps = 3
for qq in range(0,n_steps):
    
    # preallocate stiffness matrix and force vectors
    #K = np.zeros((2*np.shape(coords)[0],2*np.shape(coords)[0]))
    f = np.zeros((2*np.shape(coords)[0]))
    K = lil_matrix((2*np.shape(coords)[0],2*np.shape(coords)[0]))
    
    for ii in range(0,np.shape(emat)[0]):

        emati = np.reshape(emat[ii,:],(n+1,n+1))
        for jj in range(0,n):
            for kk in range(0,n):
                quadjk = emati[jj:(jj+2),kk:(kk+2)].flatten()
                
                for mm in range(0,4):
                    for nn in range(0,4):

                        if nn>mm:

                            # global indices of nodes 1 and 2
                            i1 = quadjk[mm]
                            i2 = quadjk[nn]

                            # coordinates of nodes 1 and 2
                            x1 = coords2[i1,0]
                            y1 = coords2[i1,1]
                            x2 = coords2[i2,0]
                            y2 = coords2[i2,1]

                            # length of "spring" connecting nodes 1 and 2
                            L = np.sqrt((x2-x1)**2 + (y2-y1)**2)

                            # spring cosine and sine (for transforming stiffness to global)
                            c = (x2-x1)/L
                            s = (y2-y1)/L

                            # coordinate transformation from spring to global
                            T = np.array([[c,s,0,0],[0,0,c,s]])
                            Kspring = kspring*np.dot(
                                    np.dot(np.transpose(T),Kmini),
                                    T)

                            # if it's a diagonal spring, make it a little weaker
                            if (mm == 1 & nn == 2) | (mm == 0 & nn == 3):
                                kspring = 1#/sqrt(2)
                                #L0mult = sqrt(2)
                            else:
                                kspring = 1
                                #L0mult = 1


                            # index of DOFs associated with current spring
                            index = [2*i1,2*i1-1,2*i2]
                            index = np.concatenate((np.arange(dpn*i1,dpn*(i1+1)),
                                                   np.arange(dpn*i2,dpn*(i2+1))),
                                                   axis=0)

                            # add current spring's stiffness into master stiffness
                            K[index[:,None],index] = K[index[:,None],index] + kspring*Kspring

                            # assume initial spring length is L0 and spring is stretched into
                            # current configuration. Compute the resulting force on the nodes
                            # due to the springs.
                            #fint = (L-L0*L0mult)*kspring*np.array([[1],[-1]])
                            fint = (L*(1/n_steps))*kspring*np.array([[1],[-1]])

                            # transform to global coordinates
                            fspring = np.dot(np.transpose(T),fint)

                            # add spring's contribution to master force vector
                            f[index[:,None]] = f[index[:,None]] + fspring
    

    Kfix = K[inotfix[:,None],inotfix]
    ffix = f[inotfix]
    
    d = np.zeros(np.shape(f))
    d[inotfix] = spsolve(Kfix.tocsr(),ffix)
    d = np.reshape(d,np.shape(coords))
    coords2 = coords2 + d
    print(qq)
                        
# plot relaxed mesh 
plotmesh(coords2,emat[:,i_perim],colorvec)

In [28]:
import math

xy = np.zeros((9,2))
D = np.zeros((3,3))
rho = 100
from mesh2D import QuadPlaneStrain

Ke,Me = QuadPlaneStrain(xy,D,rho)


LinAlgError: Singular matrix

In [17]:
print(range(0,5))


range(0, 5)
